This is from https://jovian.ai/aakashns/01-pytorch-basics and https://www.youtube.com/watch?v=m_tkL7DufPk&ab_channel=Jovian

In [1]:
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

torch over numpy to run on gpu-s for faster processing

we will use nn.linear class to initialize weights and biases of a linear model

use tensordataset to split the data into smaller chunks to run code, returns a tuple with 2 elements, in our cse inputs and targets

use dataloader to split data into bathces of predefined size; enbles shuffling and ramdom sampling of data

use nn.functional to utilize its mse_loss for the loss function

In [2]:
print(torch.__version__)

2.1.2


In [3]:
t1=torch.tensor(4.0)
t1

tensor(4.)

In [4]:
t1.dtype

torch.float32

In [5]:
t2=torch.tensor([1.,2,3,4])
t2

tensor([1., 2., 3., 4.])

In [6]:
t2.dtype

torch.float32

In [7]:
t3=torch.tensor([[5.,6],
                 [7,8],
                 [9,10]])
t3

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])

In [8]:
t4=torch.tensor([
    [[11,12,13],
     [13,14,15]],
     [[15,16,17],
      [17,18,19]]
])
t4

tensor([[[11, 12, 13],
         [13, 14, 15]],

        [[15, 16, 17],
         [17, 18, 19]]])

In [9]:
print(t1)
t1.shape

tensor(4.)


torch.Size([])

In [10]:
print(t2)
t2.shape

tensor([1., 2., 3., 4.])


torch.Size([4])

In [11]:
print(t3)
t3.shape

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])


torch.Size([3, 2])

In [12]:
print(t4)
t4.shape

tensor([[[11, 12, 13],
         [13, 14, 15]],

        [[15, 16, 17],
         [17, 18, 19]]])


torch.Size([2, 2, 3])

In [13]:
# bnasic tensor operations
x=torch.tensor(3.0)
w=torch.tensor(4.,requires_grad=True)
b=torch.tensor(5.,requires_grad=True)
x,w,b

(tensor(3.), tensor(4., requires_grad=True), tensor(5., requires_grad=True))

In [14]:
x.is_leaf

True

In [15]:
y=w*x+b
y

tensor(17., grad_fn=<AddBackward0>)

In [16]:
# y.retain_grad()
y.backward()
print('dy/dx:',x.grad)
print('dy/dw:',w.grad)
print('dy/db:',b.grad)
print('dy/dw 2:',w.grad_fn)

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)
dy/dw 2: None


In [17]:
w2=torch.tensor(42.,requires_grad=True)
b2=torch.tensor(52.,requires_grad=True)
y2=w2+b2

In [18]:
y2.backward
print(w2.grad)

None


In [19]:
t6=torch.full((3,2),42)
t6

tensor([[42, 42],
        [42, 42],
        [42, 42]])

In [20]:
t7=torch.concatenate((t3,t6))
t7

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.],
        [42., 42.],
        [42., 42.],
        [42., 42.]])

In [21]:
t8=torch.sin(t7)
t8

tensor([[-0.9589, -0.2794],
        [ 0.6570,  0.9894],
        [ 0.4121, -0.5440],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165]])

In [22]:
t9=t8.reshape(3,2,2)
t9

tensor([[[-0.9589, -0.2794],
         [ 0.6570,  0.9894]],

        [[ 0.4121, -0.5440],
         [-0.9165, -0.9165]],

        [[-0.9165, -0.9165],
         [-0.9165, -0.9165]]])

In [23]:
x=np.array([[1,2],
            [3,4.]])
x

array([[1., 2.],
       [3., 4.]])

In [24]:
y=torch.from_numpy(x)
y

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)

In [25]:
x.dtype,y.dtype

(dtype('float64'), torch.float64)

In [26]:
z=y.numpy()
z

array([[1., 2.],
       [3., 4.]])

Fundamentals of Pytorch and Numpy done.
Next progress towards linear regression znd gradient descent

Video course Timestamp 25.28 / 1.43.32  

In [27]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [28]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [29]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [30]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.5107,  0.4335, -1.6559],
        [ 1.3489,  1.1845, -0.7308]], requires_grad=True)
tensor([-0.1181,  0.6992], requires_grad=True)


In [31]:
inputs @ w.t()+b

tensor([[ -79.5552,  147.1050],
        [-114.4174,  180.9129],
        [ -82.4977,  234.3897],
        [ -94.8334,  162.1784],
        [-109.6500,  156.3294]], grad_fn=<AddBackward0>)

In [32]:
def model(x):
    return x @ w.t()+b

In [33]:
preds=model(inputs)
print(preds)

tensor([[ -79.5552,  147.1050],
        [-114.4174,  180.9129],
        [ -82.4977,  234.3897],
        [ -94.8334,  162.1784],
        [-109.6500,  156.3294]], grad_fn=<AddBackward0>)


In [34]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [35]:
diff=preds-targets
torch.sum(diff * diff)/diff.numel()

tensor(19570.8789, grad_fn=<DivBackward0>)

to get rid of negatives in diff, square numbers; * does a element wise multiplication

In [36]:
# mean squared error loss 
def mse(t1,t2):
    diff=t1-t2
    return torch.sum(diff*diff)/diff.numel()

In [37]:
loss=mse(preds,targets)
print(loss)

tensor(19570.8789, grad_fn=<DivBackward0>)


In [38]:
loss.backward()

In [39]:
print(w)
print(w.grad)

tensor([[-0.5107,  0.4335, -1.6559],
        [ 1.3489,  1.1845, -0.7308]], requires_grad=True)
tensor([[-14359.7344, -15743.5732,  -9846.1582],
        [  7413.1118,   6950.1748,   4311.0391]])


In [40]:
print(b)
print(b.grad)

tensor([-0.1181,  0.6992], requires_grad=True)
tensor([-172.3908,   84.1831])


In [41]:
with torch.no_grad():
    w-=w.grad*1e-5
    b-=b.grad*1e-5

In [42]:
w,b

(tensor([[-0.3671,  0.5909, -1.5574],
         [ 1.2747,  1.1150, -0.7739]], requires_grad=True),
 tensor([-0.1164,  0.6984], requires_grad=True))

In [43]:
preds=model(inputs)
loss=mse(preds,targets)
print(loss)

tensor(13443.9951, grad_fn=<DivBackward0>)


the grad values for w and b should be reset to zero, else the next operation will add to existing grad values

In [44]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


to train the model repeat the above operation x number of times, called epochs

In [45]:
for i in range(100):
    preds=model(inputs)
    loss=mse(preds,targets)
    loss.backward()
    with torch.no_grad():
        w-=w.grad*1e-5
        b-=b.grad*1e-5
        w.grad.zero_()
        b.grad.zero_()

In [46]:
preds=model(inputs)
loss=mse(preds,targets)
print(loss)

tensor(422.5645, grad_fn=<DivBackward0>)


In [47]:
preds

tensor([[ 60.1187,  76.5330],
        [ 71.2493,  91.8486],
        [138.8656, 143.0179],
        [ 37.0488,  71.2038],
        [ 73.5183,  83.9766]], grad_fn=<AddBackward0>)

In [48]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

xxxxxxxxx
Starting work on using Troch for gradient descent

In [49]:
# inputs are temperature, rainfall, humidity
inputs=np.array([[73,67,43],
                 [91,88,64],
                 [87,134,58],
                 [102,43,37],
                 [69,96,70],
                 [74,66,43],
                 [91,87,65],
                 [88,114,59],
                 [101,44,37],
                 [68,96,71],
                 [73,66,44],
                 [92,87,64],
                 [87,135,57],
                 [103,43,36],
                 [68,97,70]],
                 dtype='float')

# targets are apples, oranges
targets=np.array([[56,70],
                  [81,101],
                  [119,133],
                  [22,37],
                  [103,119],
                  [57,69],
                  [80,102],
                  [118,132],
                  [21,38],
                  [104,119],
                  [57,69],
                  [82,100],
                  [118,134],
                  [20,38],
                  [102,120]],
                  dtype='float')

inputs=torch.from_numpy(inputs)
targets=torch.from_numpy(targets)

In [50]:
inputs.shape

torch.Size([15, 3])

In [51]:
targets.shape

torch.Size([15, 2])

use tensordataset to split the data into smaller chunks to run code, returns a tuple with 2 elements, in our cse inputs and targets

In [52]:
train_ds=TensorDataset(inputs,targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]], dtype=torch.float64),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]], dtype=torch.float64))

use dataloader to split data into bathces of predefined size; enbles shuffling and ramdom sampling of data

In [53]:
batch_size=5
train_dl=DataLoader(train_ds,batch_size,shuffle=True)

In [54]:
for xb,yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 91.,  87.,  65.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [ 68.,  96.,  71.],
        [ 74.,  66.,  43.]], dtype=torch.float64)
tensor([[ 80., 102.],
        [ 81., 101.],
        [119., 133.],
        [104., 119.],
        [ 57.,  69.]], dtype=torch.float64)


we will use nn.linear class to initialize weights and biases of a linear model

In [55]:
# defining model usin nn.linear
model=nn.Linear(3,2,dtype=float)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.1190,  0.2176,  0.0779],
        [-0.4516,  0.0870, -0.1557]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-0.0486,  0.2190], dtype=torch.float64, requires_grad=True)


In [56]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.1190,  0.2176,  0.0779],
         [-0.4516,  0.0870, -0.1557]], dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor([-0.0486,  0.2190], dtype=torch.float64, requires_grad=True)]

In [57]:
# generate predictions using the nn.linear model above
# inputs
preds=model(inputs)
preds

tensor([[  9.1996, -33.6143],
        [ 13.2652, -43.1855],
        [ 23.2838, -36.4427],
        [  0.0592, -47.8657],
        [ 18.0910, -33.4876],
        [  8.8630, -34.1529],
        [ 13.1256, -43.4281],
        [ 18.8905, -38.7901],
        [  0.3958, -47.3270],
        [ 18.2879, -33.1916],
        [  9.0599, -33.8569],
        [ 12.9287, -43.7241],
        [ 23.4235, -36.2001],
        [ -0.1377, -48.1616],
        [ 18.4275, -32.9489]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

In [58]:
loss_fn=F.mse_loss

In [59]:
loss=loss_fn(model(inputs),targets)
print(loss)

tensor(11484.4021, dtype=torch.float64, grad_fn=<MseLossBackward0>)


use optimizer SGD, stochastic gradient descent, and a learning rate of 1e-5

In [60]:
opt=torch.optim.SGD(model.parameters(),lr=1e-5)

the function to train the model

In [61]:
# function to train the model
def fit(num_epochs,model,loss_fn,opt,train_dl):
    # repeat for x number of epochs
    for epoch in range(num_epochs):

        for xb,yb in train_dl:

            # step 1 generate predictions
            pred=model(xb)

            # step 2 calculate loss
            loss=loss_fn(pred,yb)

            # step 3 compute gradients
            loss.backward()

            # step 4 update parameters using gradients
            opt.step()

            # step 5 reset the gradients to zero
            opt.zero_grad()

        # print 
        if (epoch+1)%10==0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1,num_epochs,loss.item()))



In [62]:
fit(100,model,loss_fn,opt,train_dl)

Epoch [10/100], Loss: 219.9967
Epoch [20/100], Loss: 226.8173
Epoch [30/100], Loss: 137.0606
Epoch [40/100], Loss: 51.1669
Epoch [50/100], Loss: 22.9125
Epoch [60/100], Loss: 59.2699
Epoch [70/100], Loss: 30.7769
Epoch [80/100], Loss: 59.0479
Epoch [90/100], Loss: 15.6047
Epoch [100/100], Loss: 19.2874


In [63]:
print(w,b)

tensor([[ 0.1337,  1.3585, -0.9431],
        [ 0.5291,  1.0565, -0.7807]], requires_grad=True) tensor([-0.1087,  0.6926], requires_grad=True)


In [64]:
preds=model(inputs)
preds

tensor([[ 58.9767,  72.4733],
        [ 81.9339,  99.0622],
        [122.5056, 141.5231],
        [ 26.6617,  42.4101],
        [ 97.7375, 112.6293],
        [ 57.7910,  71.3586],
        [ 81.4367,  98.5241],
        [104.3999, 122.4837],
        [ 27.8475,  43.5248],
        [ 98.4260, 113.2058],
        [ 58.4795,  71.9351],
        [ 80.7482,  97.9475],
        [123.0028, 142.0612],
        [ 25.9733,  41.8336],
        [ 98.9232, 113.7440]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

In [65]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 119.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]], dtype=torch.float64)

lets test it for a new region

In [66]:
model(torch.tensor([[75,63,44.]],dtype=float))

tensor([[55.1957, 68.7398]], dtype=torch.float64, grad_fn=<AddmmBackward0>)